# base

> Abstract base class for managing background jobs with worker processes.

In [ ]:
#| default_exp managers.base

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import uuid
import asyncio
import multiprocessing
import queue
import threading
import atexit
import time
from abc import ABC, abstractmethod
from datetime import datetime
from typing import Dict, Any, Optional, List, Generic, TypeVar, Callable
from dataclasses import dataclass, field

from fastcore.basics import patch

from cjm_fasthtml_workers.core.config import WorkerConfig, RestartPolicy
from cjm_fasthtml_workers.core.protocol import WorkerRequestType, WorkerResponseType
from cjm_fasthtml_workers.extensions.protocols import (
    PluginRegistryProtocol,
    ResourceManagerProtocol,
    EventBroadcasterProtocol,
)

In [ ]:
#| export
# Set multiprocessing start method to 'spawn' for CUDA compatibility
# Fork mode doesn't work with CUDA (cannot re-initialize CUDA in forked subprocess)
try:
    multiprocessing.set_start_method('spawn', force=True)
except RuntimeError:
    # Already set, ignore
    pass

## BaseJob Dataclass

In [ ]:
#| export
@dataclass
class BaseJob:
    """Base class for all job types."""
    id: str  # Unique job identifier
    plugin_id: str  # Plugin identifier for this job
    status: str = "pending"  # Job status: pending, running, completed, failed, cancelled
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())  # ISO format timestamp
    started_at: Optional[str] = None  # When job started executing
    completed_at: Optional[str] = None  # When job finished
    result: Optional[Dict[str, Any]] = None  # Job result data
    error: Optional[str] = None  # Error message if failed
    metadata: Dict[str, Any] = field(default_factory=dict)  # Additional job metadata
    worker_pid: Optional[int] = None  # Process ID of worker handling this job

In [ ]:
#| export
JobType = TypeVar('JobType', bound=BaseJob)

## BaseJobManager Class

The `BaseJobManager` is an abstract base class that provides the core infrastructure for managing background jobs with worker processes. It handles:

- Worker process lifecycle (start, restart, shutdown)
- Job queuing and execution
- Result monitoring
- Resource management integration (optional)
- Event broadcasting (optional)

In [ ]:
#| export
class BaseJobManager(ABC, Generic[JobType]):
    """
    Abstract base class for managing jobs using worker processes.

    Features:
    - Jobs processed sequentially in subprocess
    - Plugin resources loaded once and reused
    - True cancellation via subprocess termination
    - Automatic worker restart based on policy
    - Isolated worker process avoids duplicating web app initialization
    - Optional streaming support for incremental results
    - Optional dependency injection for plugin registry, resource manager, and event broadcaster
    """

    def __init__(
        self,
        worker_type:str,  # Type identifier (e.g., "transcription", "llm", "image-gen")
        category:Any,  # Plugin category this manager handles
        supports_streaming:bool=False,  # Whether this manager supports streaming jobs
        worker_config:Optional[WorkerConfig]=None,  # Worker configuration (uses defaults if None)
        plugin_registry:Optional[PluginRegistryProtocol]=None,  # Optional plugin registry integration
        resource_manager:Optional[ResourceManagerProtocol]=None,  # Optional resource manager integration
        event_broadcaster:Optional[EventBroadcasterProtocol]=None,  # Optional SSE event broadcaster
    ):
        """Initialize the job manager."""
        self.worker_type = worker_type
        self.category = category
        self.supports_streaming = supports_streaming
        self.worker_config = worker_config or WorkerConfig()

        self.jobs: Dict[str, JobType] = {}
        self.results: Dict[str, Dict[str, Any]] = {}

        # Multiprocessing queues for communication with worker
        self.request_queue: Optional[multiprocessing.Queue] = None
        self.result_queue: Optional[multiprocessing.Queue] = None
        self.response_queue: Optional[multiprocessing.Queue] = None
        self.worker_process: Optional[multiprocessing.Process] = None

        # Thread for monitoring result queue
        self.result_monitor_thread: Optional[threading.Thread] = None
        self.monitor_running = False

        # Optional integrations (dependency injection)
        self.plugin_registry = plugin_registry
        self.resource_manager = resource_manager
        self.event_broadcaster = event_broadcaster

        # Worker state tracking
        self.current_plugin_name: Optional[str] = None
        self.current_plugin_id: Optional[str] = None
        self.current_config: Optional[Dict[str, Any]] = None

        # Restart tracking
        self.restart_count = 0
        self.last_restart_time: Optional[float] = None

        # Streaming support
        self._stream_buffers: Dict[str, List[str]] = {}  # job_id -> chunks

        # Worker will be started lazily on first job
        # Register cleanup on exit
        atexit.register(self.shutdown)

## Abstract Methods

Subclasses must implement these methods to customize job handling for their specific use case.

In [ ]:
#| export
@patch
@abstractmethod
def create_job(
    self:BaseJobManager,
    plugin_id:str,  # Plugin unique identifier
    **kwargs  # Domain-specific job parameters
) -> JobType:  # Created job instance
    """Factory method for creating domain-specific jobs."""
    pass

In [ ]:
#| export
@patch
@abstractmethod
def get_worker_entry_point(
    self:BaseJobManager
) -> Callable:  # Worker process entry point function
    """Return the worker process function for this manager."""
    pass

In [ ]:
#| export
@patch
@abstractmethod
def prepare_execute_request(
    self:BaseJobManager,
    job:JobType  # The job to prepare for execution
) -> Dict[str, Any]:  # Dictionary of parameters for the worker execute request
    """Convert job to worker execute request parameters."""
    pass

In [ ]:
#| export
@patch
@abstractmethod
def extract_job_result(
    self:BaseJobManager,
    job:JobType,  # The job that was executed
    result_data:Dict[str, Any]  # Raw result data from worker
) -> Dict[str, Any]:  # Formatted result for storage
    """Extract and format job result from worker response."""
    pass

## Hook Methods

These methods provide extension points for subclasses to customize behavior without overriding core functionality.

In [ ]:
#| export
@patch
def _extract_plugin_resource_identifier(
    self:BaseJobManager,
    config:Dict[str, Any]  # Plugin configuration dictionary
) -> str:  # Plugin resource identifier string
    """Extract plugin resource identifier from plugin configuration."""
    # Try common configuration keys in order of preference
    # Check 'resource_id' first, then fallback to common model-related keys for backward compatibility
    return config.get('resource_id', 
                     config.get('model_id', 
                     config.get('model', 
                     config.get('model_name', 'unknown'))))

This hook extracts a plugin resource identifier from the plugin configuration for resource tracking. The default implementation tries common keys (`resource_id`, `model_id`, `model`, `model_name`) but subclasses can override this to match their specific configuration structure.

**Example override:**
```python
def _extract_plugin_resource_identifier(self, config):
    # Custom logic for your plugin system
    return config.get('resource_path', 'unknown')
```

In [ ]:
#| export
@patch
async def _validate_resources(
    self:BaseJobManager,
    plugin_id:str,  # Plugin unique identifier
    plugin_config:Dict[str, Any]  # Plugin configuration
) -> Optional[str]:  # Error message if validation fails, None if successful
    """Validate resources before starting a job."""
    # Default implementation: no validation, always passes
    return None

This hook allows you to implement custom validation before starting a job. Return `None` if validation passes, or an error message string if it fails.

**Example override:**
```python
async def _validate_resources(self, plugin_id, plugin_config):
    # Check if another job is already running
    if self.worker_process and self.current_plugin_id:
        return "Another job is already running"
    
    # Check GPU availability
    if plugin_config.get('device') == 'cuda':
        if not torch.cuda.is_available():
            return "CUDA not available"
    
    return None  # Validation passed
```

In [ ]:
#| export
@patch
def _on_job_completed(
    self:BaseJobManager,
    job_id:str  # ID of the completed job
) -> None:
    """Hook called when a job completes successfully."""
    # Default implementation: no action
    pass

This hook is called automatically when a job completes successfully. Use it to implement post-processing logic like saving results, sending notifications, or triggering follow-up tasks.

**Example override:**
```python
def _on_job_completed(self, job_id):
    job = self.get_job(job_id)
    result = self.get_job_result(job_id)
    
    # Save result to disk
    output_path = f"results/{job_id}.json"
    with open(output_path, 'w') as f:
        json.dump(result, f)
    
    # Track completion count
    self.completed_count += 1
    
    print(f"Job {job_id} completed and saved to {output_path}")
```

## Worker Lifecycle Management

In [ ]:
#| export
@patch
def _start_worker(self:BaseJobManager):
    """Start the worker process and result monitor."""
    if self.worker_process and self.worker_process.is_alive():
        return  # Already running

    # Create fresh queues with configured sizes
    self.request_queue = multiprocessing.Queue(maxsize=self.worker_config.request_queue_size)
    self.result_queue = multiprocessing.Queue(maxsize=self.worker_config.result_queue_size)
    self.response_queue = multiprocessing.Queue(maxsize=self.worker_config.response_queue_size)

    # Start worker process using entry point from subclass
    # Using spawn method ensures clean process isolation (required for CUDA)
    worker_entry = self.get_worker_entry_point()
    self.worker_process = multiprocessing.Process(
        target=worker_entry,
        args=(self.request_queue, self.result_queue, self.response_queue)
    )
    self.worker_process.start()

    # Start result monitor thread
    self.monitor_running = True
    self.result_monitor_thread = threading.Thread(target=self._monitor_results, daemon=True)
    self.result_monitor_thread.start()

    # Register worker with resource manager if available
    if self.resource_manager:
        self.resource_manager.register_worker(
            pid=self.worker_process.pid,
            worker_type=self.worker_type
        )

    # Send initialization message with plugin configurations
    self._init_worker()

In [ ]:
#| export
@patch
def _init_worker(self:BaseJobManager):
    """Send initialization message to worker with plugin configurations."""
    # Only send plugin configs if plugin registry is available
    plugin_configs = {}
    
    if self.plugin_registry:
        # Collect all plugin configurations from registry
        plugins = self.plugin_registry.get_plugins_by_category(self.category)

        for plugin_meta in plugins:
            if plugin_meta.is_configured:
                config = self.plugin_registry.load_plugin_config(plugin_meta.get_unique_id())
                # Store by plugin name (not unique_id) since PluginManager uses name
                plugin_configs[plugin_meta.name] = config

    # Send init message with configs (worker discovers plugins)
    self.request_queue.put({
        'type': WorkerRequestType.INIT.value,
        'plugin_configs': plugin_configs
    })

In [ ]:
#| export
@patch
def _restart_worker(self:BaseJobManager):
    """Restart the worker process after an error or cancellation."""
    # Track restart
    self.restart_count += 1
    self.last_restart_time = time.time()

    # Unregister old worker from resource manager if available
    if self.resource_manager and self.worker_process:
        self.resource_manager.unregister_worker(self.worker_process.pid)

    # Stop old worker if still alive
    if self.worker_process and self.worker_process.is_alive():
        self.worker_process.terminate()
        self.worker_process.join(timeout=self.worker_config.shutdown_timeout_seconds)
        if self.worker_process.is_alive():
            self.worker_process.kill()

    # Reset state
    self.current_plugin_name = None
    self.current_plugin_id = None
    self.current_config = None

    # Start new worker
    self._start_worker()

## Result Monitoring and Handling

In [ ]:
#| export
@patch
def _monitor_results(self:BaseJobManager):
    """Monitor the result queue in a background thread."""
    while self.monitor_running:
        try:
            result = self.result_queue.get(
                timeout=self.worker_config.result_monitor_poll_interval_seconds
            )

            result_type = result.get('type')

            if result_type == WorkerResponseType.RESULT.value:
                self._handle_job_result(result)

            elif result_type == WorkerResponseType.STREAM_CHUNK.value:
                self._handle_stream_chunk(result)

            elif result_type == WorkerResponseType.READY.value:
                # Worker initialized successfully
                pass  # Could log this if needed

            elif result_type == WorkerResponseType.ERROR.value:
                print(f"Worker error: {result.get('error')}")
                # Worker had a fatal error, restart based on policy
                self._handle_worker_error()

        except queue.Empty:
            continue
        except (ValueError, OSError) as e:
            # Queue was closed (happens during worker restart/cancellation)
            # This is expected, just break out of the loop silently
            if "closed" in str(e).lower():
                break
            # For other ValueError/OSError, log and continue
            print(f"Error in result monitor: {e}")
        except Exception as e:
            print(f"Error in result monitor: {e}")

In [ ]:
#| export
@patch
def _handle_job_result(
    self:BaseJobManager, 
    result:Dict[str, Any]  # Result data from worker
):
    """Handle a job result from the worker."""
    job_id = result['job_id']

    if job_id not in self.jobs:
        return

    job = self.jobs[job_id]

    if result['status'] == 'success':
        job.status = 'completed'
        job.result = self.extract_job_result(job, result.get('data', {}))
        job.completed_at = datetime.now().isoformat()
        self.results[job_id] = {
            'status': 'success',
            'data': job.result
        }

        # Update resource manager if available - job completed, worker now idle
        if self.resource_manager and self.worker_process:
            self.resource_manager.update_worker_state(
                pid=self.worker_process.pid,
                status="idle"
            )
        
        # Call completion hook for successful jobs
        self._on_job_completed(job_id)

    elif result['status'] == 'error':
        job.status = 'failed'
        job.error = result.get('error')
        job.completed_at = datetime.now().isoformat()
        self.results[job_id] = {
            'status': 'error',
            'error': result.get('error')
        }

        # Update resource manager if available - job failed, worker now idle
        if self.resource_manager and self.worker_process:
            self.resource_manager.update_worker_state(
                pid=self.worker_process.pid,
                status="idle"
            )

In [ ]:
#| export
@patch
def _handle_stream_chunk(
    self:BaseJobManager, 
    chunk_data:Dict[str, Any]  # Chunk data from worker
):
    """Handle a streaming chunk from the worker."""
    job_id = chunk_data.get('job_id')
    chunk = chunk_data.get('chunk', '')
    is_final = chunk_data.get('is_final', False)

    if job_id not in self._stream_buffers:
        self._stream_buffers[job_id] = []

    self._stream_buffers[job_id].append(chunk)

    # If final chunk, could trigger completion
    # (actual completion still comes via RESULT message)
    if is_final:
        pass  # Placeholder for any final stream handling

In [ ]:
#| export
@patch
def _handle_worker_error(self:BaseJobManager):
    """Handle worker fatal error based on restart policy."""
    policy = self.worker_config.restart_policy

    if policy == RestartPolicy.NEVER:
        print(f"[{self.worker_type}] Worker error - restart policy is NEVER, not restarting")
        return

    if policy == RestartPolicy.ALWAYS or policy == RestartPolicy.BACKOFF:
        if self.restart_count >= self.worker_config.max_restart_attempts:
            print(f"[{self.worker_type}] Max restart attempts ({self.worker_config.max_restart_attempts}) reached")
            return

        if policy == RestartPolicy.BACKOFF:
            # Calculate backoff delay
            delay = min(
                self.worker_config.restart_backoff_base_seconds * (2 ** self.restart_count),
                self.worker_config.restart_backoff_max_seconds
            )
            print(f"[{self.worker_type}] Restarting worker after {delay:.1f}s backoff...")
            time.sleep(delay)

        self._restart_worker()

## Plugin Management Methods

In [ ]:
#| export
@patch
def get_plugin_name(
    self:BaseJobManager,
    plugin_id:str  # Plugin unique identifier
) -> Optional[str]:  # Plugin name or None
    """Get plugin name from plugin ID (requires plugin registry)."""
    if not self.plugin_registry:
        return None
        
    plugin_meta = self.plugin_registry.get_plugin(plugin_id)
    if plugin_meta:
        return plugin_meta.name
    return None

In [ ]:
#| export
@patch
async def unload_plugin(
    self:BaseJobManager,
    plugin_name:str  # Name of the plugin to unload
) -> bool:  # True if successful, False otherwise
    """Unload a plugin from the worker to free resources."""
    if not self.worker_process or not self.worker_process.is_alive():
        return False

    # Send unload request
    self.request_queue.put({
        'type': WorkerRequestType.UNLOAD.value,
        'plugin_name': plugin_name
    })

    # Wait for response (with timeout) from response queue
    try:
        response = self.response_queue.get(
            timeout=self.worker_config.unload_timeout_seconds
        )
        if (response.get('type') == WorkerResponseType.RESPONSE.value and
            response.get('request_type') == WorkerRequestType.UNLOAD.value):
            if response.get('status') == 'success':
                # Update local state
                if self.current_plugin_name == plugin_name:
                    self.current_plugin_name = None
                    self.current_plugin_id = None
                    self.current_config = None

                # Update resource manager if available
                if self.resource_manager and self.worker_process:
                    self.resource_manager.update_worker_state(
                        pid=self.worker_process.pid,
                        plugin_name=None,
                        plugin_id=None,
                        loaded_plugin_resource=None,
                        config=None,
                        status="idle"
                    )

                return True
    except queue.Empty:
        print(f"Timeout waiting for unload response for {plugin_name}")
        return False

    return False

In [ ]:
#| export
@patch
async def reload_plugin(
    self:BaseJobManager,
    plugin_name:str,  # Name of the plugin to reload
    config:Dict[str, Any]  # New configuration
) -> bool:  # True if successful, False otherwise
    """Reload a plugin with new configuration."""
    print(f"\n[Manager] Reloading plugin: {plugin_name}")
    print(f"[Manager] Device: {config.get('device', 'unknown')}")

    if not self.worker_process or not self.worker_process.is_alive():
        print("[Manager] ✗ Worker process not alive")
        return False

    # Send reload request
    print(f"[Manager] Sending reload request to worker PID {self.worker_process.pid}")
    self.request_queue.put({
        'type': WorkerRequestType.RELOAD.value,
        'plugin_name': plugin_name,
        'config': config
    })

    # Wait for response (with timeout) from response queue
    print("[Manager] Waiting for worker response...")
    try:
        response = self.response_queue.get(
            timeout=self.worker_config.reload_timeout_seconds
        )
        if (response.get('type') == WorkerResponseType.RESPONSE.value and
            response.get('request_type') == WorkerRequestType.RELOAD.value):
            if response.get('status') == 'success':
                print(f"[Manager] ✓ Reload successful")
                # Update local state
                self.current_plugin_name = plugin_name
                self.current_config = config

                # Find plugin_id if registry available
                if self.plugin_registry:
                    for plugin_meta in self.plugin_registry.get_plugins_by_category(self.category):
                        if plugin_meta.name == plugin_name:
                            self.current_plugin_id = plugin_meta.get_unique_id()
                            break

                # Update resource manager if available
                if self.resource_manager and self.worker_process:
                    # Use helper method to extract plugin resource identifier
                    resource_id = self._extract_plugin_resource_identifier(config)
                    self.resource_manager.update_worker_state(
                        pid=self.worker_process.pid,
                        plugin_name=plugin_name,
                        plugin_id=self.current_plugin_id,
                        loaded_plugin_resource=resource_id,
                        config=config,
                        status="idle"
                    )
                    print(f"[Manager] Updated resource manager state for PID {self.worker_process.pid}")

                return True
            else:
                error_msg = response.get('error', 'Unknown error')
                print(f"[Manager] ✗ Reload failed: {error_msg}")
                return False
    except queue.Empty:
        print(f"[Manager] ✗ Timeout waiting for reload response for {plugin_name}")
        return False

    return False

## Job Management Methods

In [ ]:
#| export
@patch
async def start_job(
    self:BaseJobManager,
    plugin_id:str,  # Plugin unique identifier
    **kwargs  # Domain-specific job parameters
) -> JobType:  # Created and started job
    """Start a new job."""
    # Ensure worker is running (lazy initialization)
    if not self.worker_process or not self.worker_process.is_alive():
        self._start_worker()

    # Get plugin name (requires plugin registry)
    plugin_name = self.get_plugin_name(plugin_id) if self.plugin_registry else None
    if not plugin_name and self.plugin_registry:
        raise ValueError(f"Plugin {plugin_id} not found")

    # Get plugin config if registry available
    plugin_config = None
    if self.plugin_registry:
        plugin_config = self.plugin_registry.load_plugin_config(plugin_id)
    
    # Call validation hook - subclasses can override to add custom validation
    if plugin_config:
        validation_error = await self._validate_resources(plugin_id, plugin_config)
        if validation_error:
            raise RuntimeError(f"Resource validation failed: {validation_error}")

    # Create job
    job = self.create_job(plugin_id=plugin_id, **kwargs)

    # Store job
    self.jobs[job.id] = job
    job.status = "running"
    job.started_at = datetime.now().isoformat()

    # Store worker PID for resource tracking
    if self.worker_process:
        job.worker_pid = self.worker_process.pid

    # Update manager state
    self.current_plugin_name = plugin_name
    self.current_plugin_id = plugin_id
    self.current_config = plugin_config

    # Update resource manager if available
    if self.resource_manager and self.worker_process and plugin_config:
        # Use helper method to extract plugin resource identifier
        resource_id = self._extract_plugin_resource_identifier(plugin_config)
        self.resource_manager.update_worker_state(
            pid=self.worker_process.pid,
            job_id=job.id,
            plugin_name=plugin_name,
            plugin_id=plugin_id,
            loaded_plugin_resource=resource_id,
            config=plugin_config,
            status="running"
        )

    # Send job to worker process
    execute_params = self.prepare_execute_request(job)
    self.request_queue.put({
        'type': WorkerRequestType.EXECUTE.value,
        'job_id': job.id,
        'plugin_name': plugin_name,
        **execute_params
    })

    # Broadcast job started event if broadcaster available
    if self.event_broadcaster:
        await self.event_broadcaster.broadcast(
            f"{self.worker_type}:started",
            {"job_id": job.id, "plugin_id": plugin_id}
        )

    return job

In [ ]:
#| export
@patch
async def cancel_job(
    self:BaseJobManager,
    job_id:str  # ID of the job to cancel
) -> bool:  # True if cancellation successful
    """Cancel a running job by terminating the worker process."""
    if job_id not in self.jobs:
        return False

    job = self.jobs[job_id]

    if job.status != "running":
        return False

    # Mark job as cancelled
    job.status = "cancelled"
    job.completed_at = datetime.now().isoformat()
    self.results[job_id] = {"status": "cancelled"}

    # Clean up old queues before terminating to avoid semaphore leaks
    old_request_queue = self.request_queue
    old_result_queue = self.result_queue
    old_response_queue = self.response_queue

    # Terminate the worker process (kills the running job)
    if self.worker_process and self.worker_process.is_alive():
        self.worker_process.terminate()
        self.worker_process.join(timeout=self.worker_config.shutdown_timeout_seconds)
        if self.worker_process.is_alive():
            self.worker_process.kill()

    # Close and join queues to clean up semaphores
    for q in [old_request_queue, old_result_queue, old_response_queue]:
        try:
            if q:
                q.close()
                q.join_thread()
        except:
            pass

    # Restart worker based on policy
    if self.worker_config.restart_policy in [RestartPolicy.ON_CANCELLATION, RestartPolicy.ALWAYS, RestartPolicy.BACKOFF]:
        self._restart_worker()

    # Broadcast cancellation event if broadcaster available
    if self.event_broadcaster:
        await self.event_broadcaster.broadcast(
            f"{self.worker_type}:cancelled",
            {"job_id": job_id}
        )

    return True

In [ ]:
#| export
@patch
def get_job(
    self:BaseJobManager,
    job_id:str  # Unique job identifier
) -> Optional[JobType]:  # Job object or None
    """Get a job by ID."""
    return self.jobs.get(job_id)

In [ ]:
#| export
@patch
def get_all_jobs(
    self:BaseJobManager
) -> List[JobType]:  # List of all jobs
    """Get all jobs."""
    return list(self.jobs.values())

In [ ]:
#| export
@patch
def get_job_result(
    self:BaseJobManager,
    job_id:str  # Unique job identifier
) -> Optional[Dict[str, Any]]:  # Job result or None
    """Get job result."""
    return self.results.get(job_id)

In [ ]:
#| export
@patch
def clear_completed_jobs(
    self:BaseJobManager
) -> int:  # Number of jobs cleared
    """Clear completed, failed, and cancelled jobs."""
    completed = [
        job_id for job_id, job in self.jobs.items()
        if job.status in ['completed', 'failed', 'cancelled']
    ]

    for job_id in completed:
        del self.jobs[job_id]
        if job_id in self.results:
            del self.results[job_id]
        if job_id in self._stream_buffers:
            del self._stream_buffers[job_id]

    return len(completed)

## Utility Methods

## Usage Example

To use `BaseJobManager`, create a concrete subclass that implements the required abstract methods. Here's a minimal example structure:

```python
from dataclasses import dataclass
from cjm_fasthtml_workers.managers.base import BaseJobManager, BaseJob

@dataclass
class MyJob(BaseJob):
    """Custom job type with domain-specific fields."""
    input_text: str = ""
    
class MyJobManager(BaseJobManager[MyJob]):
    """Concrete job manager implementation."""
    
    def create_job(self, plugin_id: str, **kwargs) -> MyJob:
        """Create a job instance."""
        return MyJob(
            id=str(uuid.uuid4()),
            plugin_id=plugin_id,
            input_text=kwargs.get('input_text', '')
        )
    
    def get_worker_entry_point(self) -> Callable:
        """Return the worker function."""
        return my_worker_process
    
    def prepare_execute_request(self, job: MyJob) -> Dict[str, Any]:
        """Convert job to execution parameters."""
        return {'text': job.input_text}
    
    def extract_job_result(self, job: MyJob, result_data: Dict[str, Any]) -> Dict[str, Any]:
        """Format the result."""
        return result_data

# Create and use the manager
manager = MyJobManager(
    worker_type="text_processing",
    category="processing",
    worker_config=WorkerConfig()
)

# Start a job
job = await manager.start_job(plugin_id="my-plugin", input_text="Hello World")
```

For a complete working example, see the `demo_app.py` file in the repository.

In [ ]:
#| export
@patch
async def broadcast_event(
    self:BaseJobManager,
    event_type:str,  # Event type identifier
    data:Dict[str, Any]  # Event data payload
):
    """Broadcast an event to all connected SSE clients (requires event broadcaster)."""
    if self.event_broadcaster:
        await self.event_broadcaster.broadcast(event_type, data)

In [ ]:
#| export
@patch
def check_streaming_support(
    self:BaseJobManager,
    plugin_id:str  # Plugin unique identifier
) -> bool:  # True if streaming supported
    """Check if a plugin supports streaming."""
    # This would need to be implemented by subclasses if needed
    # For now, return the manager's streaming support flag
    return self.supports_streaming

In [ ]:
#| export
@patch
def shutdown(self:BaseJobManager):
    """Shutdown the manager and cleanup resources."""
    # Stop result monitor
    self.monitor_running = False
    if self.result_monitor_thread:
        self.result_monitor_thread.join(timeout=2)

    # Unregister worker from resource manager if available
    if self.resource_manager and self.worker_process:
        self.resource_manager.unregister_worker(self.worker_process.pid)

    # Stop worker process
    if self.worker_process and self.worker_process.is_alive():
        # Try graceful shutdown first
        try:
            self.request_queue.put({'type': WorkerRequestType.STOP.value})
            self.worker_process.join(timeout=self.worker_config.shutdown_timeout_seconds)
        except:
            pass

        # Force kill if still alive
        if self.worker_process.is_alive():
            self.worker_process.terminate()
            self.worker_process.join(timeout=2)
            if self.worker_process.is_alive():
                self.worker_process.kill()

    # Clean up queues to prevent semaphore leaks
    for q in [self.request_queue, self.result_queue, self.response_queue]:
        try:
            if q:
                q.close()
                q.join_thread()
        except:
            pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()